# Two Independent Variables

Fit to an equation of the form ([ref](https://math.stackexchange.com/questions/3975004/fit-sum-of-multivariate-exponentials)):

$$
z(x,y)=ae^{bx}y+ce^{dx}
$$
We  want to find$ $a,$ $b,$ $c,$ $d values given values of$ $x,$ $y and$ $z from measurement data. 
This specific case is a lucky one because the term that depends on the independent variable $y$ affects linearly the non-linear exponentials that depend on $x$. So the problem remains linear, and you can still apply [the method I proposed here](https://math.stackexchange.com/questions/1428566/fit-sum-of-exponentials/) (with some minor changes).

First we have to store the data $z(x,y)$ in a (tall) column vector, putting each $y$ sample head-to-tail with each other. Assuming we have $nx$ number of samples in $x$, and $ny$ number of samples in $y$:

$$
z = \left[
\begin{array}{c}
z(x, y_1) \\
z(x, y_2) \\
\vdots \\
z(x, y_{ny})
\end{array}
\right]
$$

Now we need to equate the right hand side of, the equation according to the proposed method. For all $ny$ samples in $y$, the exponental terms ("lambdas") are the same, but the terms that multiply $x$ and the constant are not, so the matrix should have the form:

$$
Z = \left[
\begin{array}{c}
\int{z(x, y_1)} & \int^2{z(x, y_1)} & x & 1 & 0 & 0 & \dots & 0 & 0 \\
\int{z(x, y_2)} & \int^2{z(x, y_2)} & 0 & 0 & x & 1 & \dots & 0 & 0 \\
\vdots \\
\int{z(x, y_{ny})} & \int^2{z(x, y_{ny})} & 0 & 0 & 0 & 0 & \dots & x & 1
\end{array}
\right]
$$

So now we have:

$$
z = Z A
$$

So:

$$
A = (Z^T Z)^{-1}Z^T z 
$$

$$
\lambda = eig\left(
\left[
\begin{array}{c}
A(1) & A(2) \\
1 & 0
\end{array}
\right]
\right) 
$$

The rest follows by the proposed method.


---

# Example

## Import [PyArma](https://pyarma.sourceforge.io/)

In [1]:
from pyarma import *
from pyarma_utils import cumtrapz, join_horizontal

## Generate data to fit

Independent variable $x$

In [2]:
dx = 0.02
x = regspace(dx, dx, 1.5)
nx = x.n_elem

Independent variable $y$

In [3]:
dy = 0.05
y = regspace(dy, dy, 1.5)
ny = y.n_elem

Dependent variable $z(x,y)$

Stored in column vector head-to-tail for each $y[k]$

In [4]:
z = zeros(nx*ny, 1)
for k in range(0, ny):
    z_start = nx*k
    z_end   = nx*(k+1)-1
    z[z_start:z_end,:] = 5*exp(0.5*x)*y[k] + 4*exp(-3*x)

## Compute exponentials lambdas

Build least squares matrix $Z$ to obtain the lambdas

In [5]:
Z = zeros(nx*ny, 2+2*ny)
for k in range(0, ny):
    z_start = nx*k
    z_end   = nx*(k+1)-1
    # calculate integrals of z(y[k]) wrt x, for each kth sample in y
    iz1 = cumtrapz(x, z[z_start:z_end,:])
    iz2 = cumtrapz(x, iz1)
    # store in appropriate location
    Z[z_start:z_end, 0:1] = join_horizontal(iz1, iz2)
    Z[z_start:z_end, 2+2*k:3+2*k] = join_horizontal(x, ones(size(x)))

Solve the least squares and eigenvalue problems

In [6]:
A = pinv(Z)*z
lambdas = eig_gen(mat([
    [A[0], A[1]],
    [1, 0]
]))[0]
lambdas = real(lambdas)
lambdas.print("lambdas = ")

lambdas = 
  -2.9991
   0.5000


## Compute exponentials multipliers

Build least squares matrix $X$ and solve the least squares problem to obtain the multipliers

In [7]:
X = zeros(nx*ny, 2)
for k in range(0, ny):
    x_start = nx*k
    x_end   = nx*(k+1)-1
    X[x_start:x_end, :] = join_horizontal(exp(lambdas[1]*x)*y[k], exp(lambdas[0]*x))
# solve the least squares problem
P = pinv(X)*z;
P = real(P)
P.print("P = ")

P = 
   4.9999
   3.9995


## Discussion

Note that in the "Compute exponentials multipliers" section of the code,awe I used t expession

```python
join_horizontal( exp(lambdas[1]*x)*y[k], exp(lambdas[0]*x) )
```

for the fit, because we already knew in advance which exponential gets multiplied by $y(k)$. But actually we don't know at this point which of the two $exp(lambda(?)*x)$ is the one that gets multiplied by $y(k)$. So actually in practice, we would need to also try:

```python
join_horizontal( exp(lambdas[1]*x), exp(lambdas[0]*x)*y[k] )
```

And see which one makes the best fit. best fit.